# **CA4 - Part one**

In [1]:
!pip install transformers

Import required packages


In [2]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW
from transformers import BertModel
from transformers import TrainingArguments
from transformers import Trainer
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle


import torch
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

Drive Mounted

In [3]:
data = "/content/drive/MyDrive/Data_Colab/NLP_CA4_Data"

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/Data_Colab/NLP_CA4_Data/Part2 - Entailments/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Data_Colab/NLP_CA4_Data/Part2 - Entailments/test.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Data_Colab/NLP_CA4_Data/Part2 - Entailments/dev.csv')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [6]:
train_df.head()

,Unnamed: 0,sent1,sent2,label,source
0,325,زنان به قدری بخش بزرگی از نیروی کار را تشکیل م...,مردان بخش عظیمی از نیروی کار هستند بنابراین تن...,c,translation-train
1,407,سالها است که کنگره در تلاش است تا اثربخشی مدیر...,کنگره بودجه ویژه ای برای مدیریت اطلاعات و فناو...,n,translation-train
2,762,سرامیک‌های زیست خنثی پس از قرارگیری در بدن میز...,خواص فیزیکی سرامیک‌ها قابل اندازه گیری است.,n,natural-wiki
3,575,دولت از هیچ قانونی که منجر به کاهش چشمگیر توان...,قانونی که باعث کاهش استفاده از زغال سنگ به عنو...,e,translation-train
4,7342,روش‌ها و الگوریتم‌های بهینه‌سازی به دو دسته ال...,آمار در دروس مدیریتی نقش مهمی را بازی میکند.,n,natural-wiki


In [7]:
#Missing values in training set
train_df.isnull().sum()
#Missing values in test set
test_df.isnull().sum()

Unnamed: 0    0
sent1         0
sent2         0
label         0
source        0
dtype: int64

In [8]:
# Entailment text
print("Entailment example :",train_df[train_df['label']=='e']['sent1'].values[0], train_df[train_df['label']=='e']['sent2'].values[0])
#Contradiction_text
print("Contradiction example :",train_df[train_df['label']=='c']['sent1'].values[0], train_df[train_df['label']=='c']['sent2'].values[0])
#neutral_text
print("Neutral example  :",train_df[train_df['label']=='n']['sent1'].values[0] , train_df[train_df['label']=='n']['sent2'].values[0])

Entailment example : دولت از هیچ قانونی که منجر به کاهش چشمگیر توانایی کشورمان در استفاده از زغال سنگ به عنوان منبع اصلی برق فعلی و آینده شود ، پشتیبانی نخواهد کرد. قانونی که باعث کاهش استفاده از زغال سنگ به عنوان منبع انرژی شود ، توسط دولت پشتیبانی نمی شود.
Contradiction example : زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد. مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.
Neutral example  : سالها است که کنگره در تلاش است تا اثربخشی مدیریت اطلاعات و فناوری را در دولت فدرال افزایش دهد. کنگره بودجه ویژه ای برای مدیریت اطلاعات و فناوری در دولت فدرال دارد.


In [9]:
train_df["label"] = train_df["label"].map({"e": 2, "c": 1,'n':0})
test_df["label"] = test_df["label"].map({"e": 2, "c": 1,'n':0})
valid_df["label"] = valid_df["label"].map({"e": 2, "c": 1,'n':0})

In [10]:
train_df = train_df.fillna({"label": 0})
test_df = test_df.fillna({"label": 0})
valid_df = valid_df.fillna({"label": 0})

In [11]:
train_df.label = train_df.label.astype('int')
test_df.label = test_df.label.astype('int')
valid_df.label = valid_df.label.astype('int')

In [12]:
train_df.sample(n=100).sent2.values

array(['آمریکایی ها فاقد احساس درست و غلط هستند. آنها عموماً کودن هستند.',
       'دوک ونیز، با تجدید قراردادهایش باعث شد نفرت  ونیز از دولت بیزانس کاهش یابد.',
       'هم اکنون هیچ روش و استراتژی ای وجود ندارد که بتواند به CIO ها کمک کند تا نیازهای تجاری را برآورده سازند.',
       'مسئله اقتصادی نیست.',
       'روشهای زندگی قدیمی جمعیت بومی تقریباً کاملاً از بین رفته است.',
       'نه من در غرب ، نیومکزیکو هستم.', '--',
       'همه بازدید کنندگان تابستانی باغ وحش ادینبورگ، رژه پنگوئن ها را می بینند.',
       'کوبا آماده\u200cی از سر گرفتن روابط با آمریکا است.',
       'مشکل ما با آداب و رسوم هدیه دادن، افرادی هستند که نمی دانند چگونه هدیه های مناسبی انتخاب کنند.',
       'این گروه مخالف تخریب آثار هنری و باستان\u200c شناسی بودند.',
       'گربه های من احتمالاً می توانند این کار را انجام دهند زیرا باهوش هستند.',
       'من سعی کردم در محل کار دامن کوتاه بپوشم ، اما یاد گرفتم که ایده خوبی نیست.',
       'به خاطر تلفات بسیار زیادی که از هر دو سپاه رخ داد، این جنگ به بوستان کشتگان معروف گ

In [13]:
'در مهر ۱۳۹۲، برخی رسانه\u200cها با نگاهی به مقاله "ادعای مضحک امروز" نوشته شده در "سایت اسراییل کول" از مسلمان شدن روآن اتکینسون خبر دادند.'.replace('\u200c' , ' ')

'در مهر ۱۳۹۲، برخی رسانه ها با نگاهی به مقاله "ادعای مضحک امروز" نوشته شده در "سایت اسراییل کول" از مسلمان شدن روآن اتکینسون خبر دادند.'

Handling half space

In [14]:
##train set
train_df.sent1 = train_df.sent1.str.replace('\u200c' , ' ')
train_df.sent2 = train_df.sent2.str.replace('\u200c' , ' ')
##test set
test_df.sent1 = test_df.sent1.str.replace('\u200c' , ' ')
test_df.sent2 = test_df.sent2.str.replace('\u200c' , ' ')
##val set
valid_df.sent1 = valid_df.sent1.str.replace('\u200c' , ' ')
valid_df.sent2 = valid_df.sent2.str.replace('\u200c' , ' ')

Clean dataframe

In [15]:
train_data = train_df.drop(columns=["Unnamed: 0","source"])
test_data = test_df.drop(columns=["Unnamed: 0","source"])
valid_data = valid_df.drop(columns=["Unnamed: 0","source"])

Split dataset - Train and test Args

In [16]:

X_train = train_data[['sent1','sent2']]
y_train = train_data['label']

X_test = test_data[['sent1','sent2']]
y_test = test_data['label']

X_eval = valid_df[['sent1','sent2']]
y_eval = valid_df['label']

Make lists from independent variables

In [17]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

X_eval = X_eval.values.tolist()
y_eval = y_eval.values.tolist()

In [18]:
X_train

[['زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد.',
  'مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.'],
 ['سالها است که کنگره در تلاش است تا اثربخشی مدیریت اطلاعات و فناوری را در دولت فدرال افزایش دهد.',
  'کنگره بودجه ویژه ای برای مدیریت اطلاعات و فناوری در دولت فدرال دارد.'],
 ['سرامیک های زیست خنثی پس از قرارگیری در بدن میزبان خواص فیزیکی و مکانیکی خود را حفظ می کند.',
  'خواص فیزیکی سرامیک ها قابل اندازه گیری است.'],
 ['دولت از هیچ قانونی که منجر به کاهش چشمگیر توانایی کشورمان در استفاده از زغال سنگ به عنوان منبع اصلی برق فعلی و آینده شود ، پشتیبانی نخواهد کرد.',
  'قانونی که باعث کاهش استفاده از زغال سنگ به عنوان منبع انرژی شود ، توسط دولت پشتیبانی نمی شود.'],
 ['روش ها و الگوریتم های بهینه سازی به دو دسته الگوریتمهای دقیق (exact) و الگوریتم های تقریبی (approximate algorithms) تقسیم بندی می شوند.',
  'آمار در دروس مدیریتی نقش مهمی را بازی میکند. '],
 ['گرچه 

Labels Distributions

In [19]:
entailments_count = train_data[train_data['label'] == 2].shape[0]
contract_count = train_data[train_data['label'] == 1].shape[0]
neutral_count = train_data[train_data['label'] == 0].shape[0]

train_distribution = {'entailments': [entailments_count], 'contract': [contract_count], 'neutral': [neutral_count]}
train_distribution = pd.DataFrame(train_distribution)

train_distribution

,entailments,contract,neutral
0,304,205,246


## Configuration values

In [20]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128

EPOCHS = 10
EVERY_EPOCH = 500
LEARNING_RATE = 1e-5

model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)
# model = XLMRobertaForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased', num_labels=3)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

### Create Data loader

In [21]:
class EntailmentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

XLM Roberta Tokenizer

In [38]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base',
                                                    num_labels=3,
                                                    loss_function_params={"weight": [0.75, 0.25]}
                                                    )
#tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model in cache at /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
creating metadata file for /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpuftmprmv


Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
creating metadata file for /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file h

In [24]:
train_encoding = tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LEN)

In [25]:
eval_encoding = tokenizer(X_eval, padding=True, truncation=True, max_length=MAX_LEN)

In [26]:
test_encoding = tokenizer(X_test, padding=True, truncation=True, max_length=MAX_LEN)

In [27]:
train_dataset = EntailmentDataset(train_encoding, y_train)
eval_dataset = EntailmentDataset(eval_encoding, y_eval)
test_dataset = EntailmentDataset(test_encoding, y_test)

## Fine tuning

## XLM Robera

In [27]:
# batch_sizes = [4, 8, 16, 32, 64, 128]
# learning_rates = [1e-6, 5e-6, 8e-6, 1e-5, 2e-5, 5e-5]
# epochs = [5, 10, 15, 20, 30]
# embedding_lenths = [64, 128, 256, 512]
# beta1 = [0.9, 0.5, 0.1]
# beta2 = [0.999, 0.5, 0.001]

In [28]:
# training_args = TrainingArguments("test_trainer")

training_args = TrainingArguments(
    output_dir='output',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    eval_steps=EVERY_EPOCH,
    metric_for_best_model='accuracy',
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    load_best_model_at_end=True,
    save_total_limit = 1,
    logging_steps = 20

)
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate

In [29]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

In [49]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
training_metrics = trainer.train()

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 270
  Batch size = 128


{'epoch': 10.0,
 'eval_accuracy': 0.34444444444444444,
 'eval_f1_score': 0.24362624601364058,
 'eval_loss': 1.1117578744888306,
 'eval_runtime': 1.756,
 'eval_samples_per_second': 153.76,
 'eval_steps_per_second': 1.708}

In [33]:
pin_memory=False
preds = trainer.predict(test_dataset=test_dataset)
preds

***** Running Prediction *****
  Num examples = 1675
  Batch size = 128


PredictionOutput(predictions=array([[-0.03201459,  0.03097646,  0.35383135],
       [ 0.29030406, -0.11607438,  0.39120656],
       [ 0.44012144, -0.22175242,  0.38725245],
       ...,
       [ 0.25291538, -0.04317278,  0.30814347],
       [ 0.24994732, -0.21832702,  0.45847595],
       [ 0.04218419,  0.00180826,  0.3609193 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 1.0893194675445557, 'test_accuracy': 0.4137313432835821, 'test_f1_score': 0.3090069844992649, 'test_runtime': 10.3726, 'test_samples_per_second': 161.483, 'test_steps_per_second': 1.35})

In [34]:
preds.metrics

{'test_accuracy': 0.4137313432835821,
 'test_f1_score': 0.3090069844992649,
 'test_loss': 1.0893194675445557,
 'test_runtime': 10.3726,
 'test_samples_per_second': 161.483,
 'test_steps_per_second': 1.35}

## **Pars Bert**

In [22]:
tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-zwnj-base')

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [3]:
!rm -rf sample_data/

In [31]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128

EPOCHS = 10
EVERY_EPOCH = 500
LEARNING_RATE = 5e-5

model = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-zwnj-base', num_labels=3)

loading configuration file https://huggingface.co/HooshvareLab/bert-fa-zwnj-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ba5343a1268e870ceacba2041556e22a4480d6af08e330b0650f96c96a5e50b4.967063c40b5c987b6e0d2a6d1944963280b9a53e5821fcb5379494a07dc9408d
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-zwnj-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "a

In [63]:
CUDA_LAUNCH_BLOCKING=1

In [32]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [33]:
training_metrics = trainer.train()

***** Running training *****
  Num examples = 755
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 120


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,No log,1.078940,0.374074,0.347452
2,1.063200,1.103733,0.362963,0.349403
3,1.063200,1.173901,0.348148,0.339735
4,0.773600,1.369666,0.381481,0.361122
5,0.378500,1.556215,0.366667,0.354086
6,0.378500,1.686088,0.411111,0.410992
7,0.124100,1.903781,0.400000,0.398577
8,0.124100,2.109486,0.396296,0.397555
9,0.042600,2.292543,0.392593,0.392175
10,0.023300,2.337550,0.388889,0.388582


***** Running Evaluation *****
  Num examples = 270
  Batch size = 128
Saving model checkpoint to output/checkpoint-12
Configuration saved in output/checkpoint-12/config.json
Model weights saved in output/checkpoint-12/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 270
  Batch size = 128
Saving model checkpoint to output/checkpoint-24
Configuration saved in output/checkpoint-24/config.json
Model weights saved in output/checkpoint-24/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 270
  Batch size = 128
Saving model checkpoint to output/checkpoint-36
Configuration saved in output/checkpoint-36/config.json
Model weights saved in output/checkpoint-36/pytorch_model.bin
Deleting older checkpoint [output/checkpoint-24] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 270
  Batch size = 128
Saving model checkpoint to output/checkpoint-48
Configuration saved in output/checkpoint-48/config.json
Model weights saved in output/checkp

In [34]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 270
  Batch size = 128


{'epoch': 10.0,
 'eval_accuracy': 0.4111111111111111,
 'eval_f1_score': 0.4109915529564652,
 'eval_loss': 1.6860878467559814,
 'eval_runtime': 1.9211,
 'eval_samples_per_second': 140.544,
 'eval_steps_per_second': 1.562}

In [35]:
pin_memory=False
preds = trainer.predict(test_dataset=test_dataset)
preds

***** Running Prediction *****
  Num examples = 1675
  Batch size = 128


PredictionOutput(predictions=array([[ 0.55590296, -0.21365398,  0.03142007],
       [-0.29242116,  0.43592358,  0.25376225],
       [-1.1852447 ,  0.46819693,  1.4328766 ],
       ...,
       [ 1.4528607 , -1.8916134 ,  0.6658645 ],
       [ 0.33987936, -0.6867575 ,  1.3570824 ],
       [-2.5281432 ,  1.2332146 ,  1.854781  ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 1.8001621961593628, 'test_accuracy': 0.37910447761194027, 'test_f1_score': 0.37499179789530696, 'test_runtime': 11.5762, 'test_samples_per_second': 144.693, 'test_steps_per_second': 1.209})

In [37]:
preds.metrics

{'test_accuracy': 0.3701492537313433,
 'test_f1_score': 0.2628860309874286,
 'test_loss': 1.1056609153747559,
 'test_runtime': 11.3612,
 'test_samples_per_second': 147.432,
 'test_steps_per_second': 1.232}